In [ ]:
import pandas as pd
import numpy as np
import re
import pickle
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

# --- CONFIGURAÇÕES DE CAMINHO ---
ARQUIVO_MODELO = "vetorizador.pkl"
ARQUIVO_CLUSTERS = "base_clusters.csv"
ARQUIVO_CENTROIDES = "centroides.pkl"

# --- 1. FUNÇÃO DE LIMPEZA (O seu tratamento de texto) ---
def limpar_texto(texto):
    """Remove caracteres especiais, acentos e padroniza para minúsculo."""
    if not isinstance(texto, str): return ""
    texto = texto.lower()
    # Mantém apenas letras de 'a' a 'z' e espaços
    texto = re.sub(r'[^a-z\s]', '', texto)
    return texto.strip()

# --- 2. FUNÇÕES DE CARGA E SALVAMENTO ---
def carregar_estado():
    """Carrega os dados salvos anteriormente para continuar de onde parou."""
    vetorizador = None
    clusters_df = pd.DataFrame()
    centroides_lista = []

    if os.path.exists(ARQUIVO_MODELO):
        with open(ARQUIVO_MODELO, 'rb') as f:
            vetorizador = pickle.load(f)
    
    if os.path.exists(ARQUIVO_CLUSTERS):
        clusters_df = pd.read_csv(ARQUIVO_CLUSTERS)
        # Converte o texto da coluna 'cpfs_unicos' de volta para um conjunto (set)
        clusters_df['cpfs_unicos'] = clusters_df['cpfs_unicos'].apply(eval).apply(set)
    
    if os.path.exists(ARQUIVO_CENTROIDES):
        with open(ARQUIVO_CENTROIDES, 'rb') as f:
            centroides_lista = pickle.load(f)
            
    return vetorizador, clusters_df, centroides_lista

def salvar_estado(vetorizador, clusters_df, centroides_lista):
    """Salva os dados atuais em arquivos para persistência."""
    with open(ARQUIVO_MODELO, 'wb') as f:
        pickle.dump(vetorizador, f)
    
    with open(ARQUIVO_CENTROIDES, 'wb') as f:
        pickle.dump(centroides_lista, f)
        
    # No CSV, salvamos o set de CPFs como lista para ser legível
    df_temp = clusters_df.copy()
    df_temp['cpfs_unicos'] = df_temp['cpfs_unicos'].apply(list)
    df_temp.to_csv(ARQUIVO_CLUSTERS, index=False)

# --- 3. LÓGICA PRINCIPAL (INCREMENTAL) ---
def processar_dados(df_novo, limiar=0.60, min_cpfs=3):
    # Tenta carregar o que já existe
    vetorizador, clusters_df, centroides_lista = carregar_estado()

    # Se o vetorizador não existir, cria um novo (Treino inicial)
    if vetorizador is None:
        print("Treinando vetorizador pela primeira vez...")
        try:
            stops = stopwords.words('portuguese')
        except:
            nltk.download('stopwords')
            stops = stopwords.words('portuguese')
        
        vetorizador = TfidfVectorizer(stop_words=stops, max_features=5000)
        textos_treino = [limpar_texto(t) for t in df_novo['texto']]
        vetorizador.fit(textos_treino)

    # Prepara os textos novos
    textos_limpos = [limpar_texto(t) for t in df_novo['texto']]
    matriz_novos_vetores = vetorizador.transform(textos_limpos)
    
    resultados = []

    # Percorre cada texto novo
    for i in range(matriz_novos_vetores.shape[0]):
        vetor_atual = matriz_novos_vetores[i]
        cpf_atual = str(df_novo.iloc[i]['cpf'])
        id_atual = df_novo.iloc[i]['id']
        
        melhor_cluster_idx = -1
        maior_score = 0.0

        # Compara com os centroides existentes
        if len(centroides_lista) > 0:
            matriz_centroides = np.vstack(centroides_lista)
            sims = cosine_similarity(vetor_atual, matriz_centroides).flatten()
            melhor_cluster_idx = np.argmax(sims)
            maior_score = sims[melhor_cluster_idx]

        # SE encontrar similaridade >= 60%
        if melhor_cluster_idx != -1 and maior_score >= limiar:
            # Pega o ID do cluster encontrado
            c_id = clusters_df.iloc[melhor_cluster_idx]['cluster_id']
            
            # Atualiza metadados (CPFs e Quantidade)
            clusters_df.at[melhor_cluster_idx, 'cpfs_unicos'].add(cpf_atual)
            clusters_df.at[melhor_cluster_idx, 'quantidade_mensagens'] += 1
            
            # Verifica se virou influenciador
            if len(clusters_df.at[melhor_cluster_idx, 'cpfs_unicos']) >= min_cpfs:
                clusters_df.at[melhor_cluster_idx, 'e_influenciador'] = True
            
            # Recalcula o centroide (Média móvel para o grupo evoluir)
            n = clusters_df.at[melhor_cluster_idx, 'quantidade_mensagens']
            antigo_centroide = centroides_lista[melhor_cluster_idx]
            novo_centroide = (antigo_centroide * (n-1) + vetor_atual.toarray()) / n
            centroides_lista[melhor_cluster_idx] = novo_centroide
            
            cluster_final = c_id
        else:
            # SE NÃO encontrar, cria um cluster NOVO
            cluster_final = len(clusters_df) + 1
            novo_reg = {
                'cluster_id': cluster_final,
                'quantidade_mensagens': 1,
                'cpfs_unicos': {cpf_atual},
                'e_influenciador': False
            }
            clusters_df = pd.concat([clusters_df, pd.DataFrame([novo_reg])], ignore_index=True)
            centroides_lista.append(vetor_atual.toarray())
            maior_score = 1.0 # Similaridade com ele mesmo ao criar

        # Adiciona ao relatório de saída
        e_influ = clusters_df[clusters_df['cluster_id'] == cluster_final]['e_influenciador'].values[0]
        resultados.append({
            'id': id_atual,
            'cluster_atribuido': cluster_final,
            'similaridade': round(float(maior_score), 4),
            'influenciador': 1 if e_influ else 0
        })

    # Salva tudo nos arquivos para a próxima rodada
    salvar_estado(vetorizador, clusters_df, centroides_lista)
    
    # Retorna o DataFrame com as novas colunas
    return pd.merge(df_novo, pd.DataFrame(resultados), on='id')

# --- EXECUÇÃO ---
if __name__ == "__main__":
    # Dados de exemplo (Simulando o df_influ)
    meus_dados = pd.DataFrame({
        'id': [1, 2, 3],
        'texto': ["Problema com o pix fora do ar", "Pix nao funciona", "Meu cartao atrasou"],
        'cpf': ['111', '222', '333']
    })

    # Chama a função principal
    resultado_final = processar_dados(meus_dados)
    print(resultado_final)

In [15]:
import pandas as pd
import numpy as np
import re
import pickle
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# --- CONFIGURAÇÕES DE ARQUIVOS ---
CAMINHO_MODELO = "vetorizador_tfidf.pkl"
CAMINHO_CLUSTERS = "base_clusters.csv"
CAMINHO_CENTROIDES = "centroides_matriz.pkl"

class ProcessadorTexto:
    """Classe responsável pela limpeza padronizada dos textos"""
    @staticmethod
    def limpar(texto):
        if not isinstance(texto, str): return ""
        texto = texto.lower()
        # Remove acentos e caracteres especiais
        texto = re.sub(r'[^a-z\s]', '', texto)
        return texto.strip()

class DetectorIncrementalInfluenciador:
    def __init__(self, similaridade_minima=0.60, minimo_cpfs=3):
        self.limiar = similaridade_minima
        self.min_cpfs = minimo_cpfs
        self.vetorizador = None
        self.clusters_df = pd.DataFrame() # Memória dos metadados dos clusters
        self.centroides_lista = []        # Memória dos vetores (centroides)
        
        # Tenta carregar dados existentes ao iniciar
        self._carregar_estado()

    def _carregar_estado(self):
        """Carrega os arquivos salvos se eles existirem"""
        if os.path.exists(CAMINHO_MODELO):
            with open(CAMINHO_MODELO, 'rb') as f:
                self.vetorizador = pickle.load(f)
        
        if os.path.exists(CAMINHO_CLUSTERS):
            self.clusters_df = pd.read_csv(CAMINHO_CLUSTERS)
            # Converte a string de CPFs de volta para um conjunto (set)
            self.clusters_df['cpfs_unicos'] = self.clusters_df['cpfs_unicos'].apply(eval).apply(set)
        
        if os.path.exists(CAMINHO_CENTROIDES):
            with open(CAMINHO_CENTROIDES, 'rb') as f:
                self.centroides_lista = pickle.load(f)

    def _salvar_estado(self):
        """Persiste os dados em disco para a próxima execução"""
        with open(CAMINHO_MODELO, 'wb') as f:
            pickle.dump(self.vetorizador, f)
        
        with open(CAMINHO_CENTROIDES, 'wb') as f:
            pickle.dump(self.centroides_lista, f)
            
        # Para salvar no CSV, convertemos o set de CPFs em lista (string)
        df_para_salvar = self.clusters_df.copy()
        df_para_salvar['cpfs_unicos'] = df_para_salvar['cpfs_unicos'].apply(list)
        df_para_salvar.to_csv(CAMINHO_CLUSTERS, index=False)

    def inicializar_modelo(self, textos_treino):
        """Treina o TF-IDF pela primeira vez (Cold Start)"""
        from nltk.corpus import stopwords
        import nltk
        try:
            pt_stops = stopwords.words('portuguese')
        except:
            nltk.download('stopwords')
            pt_stops = stopwords.words('portuguese')

        self.vetorizador = TfidfVectorizer(stop_words=pt_stops, max_features=5000)
        textos_limpos = [ProcessadorTexto.limpar(t) for t in textos_treino]
        self.vetorizador.fit(textos_limpos)
        print("✅ Modelo TF-IDF inicializado e treinado.")

    def processar_lote_diario(self, novos_dados):
        """
        Recebe um DataFrame com ['id', 'texto', 'cpf'].
        Retorna o mesmo DataFrame com as colunas de cluster e influencer.
        """
        textos_limpos = [ProcessadorTexto.limpar(t) for t in novos_dados['texto']]
        matriz_vetores = self.vetorizador.transform(textos_limpos)
        
        resultados_finais = []

        # Processa cada linha do novo lote
        for i in range(matriz_vetores.shape[0]):
            vetor_atual = matriz_vetores[i]
            cpf_atual = str(novos_dados.iloc[i]['cpf'])
            id_origem = novos_dados.iloc[i]['id']
            
            melhor_cluster_id = None
            maior_similaridade = 0.0
            
            # 1. Comparação com clusters existentes
            if len(self.centroides_lista) > 0:
                # Transforma lista de centroides em matriz para cálculo rápido
                matriz_centroides = np.vstack(self.centroides_lista)
                similaridades = cosine_similarity(vetor_atual, matriz_centroides).flatten()
                
                indice_melhor = np.argmax(similaridades)
                if similaridades[indice_melhor] >= self.limiar:
                    maior_similaridade = similaridades[indice_melhor]
                    melhor_cluster_id = self.clusters_df.iloc[indice_melhor]['cluster_id']

            # 2. Lógica de Atribuição ou Criação
            if melhor_cluster_id is not None:
                # Atualiza cluster existente
                idx = self.clusters_df[self.clusters_df['cluster_id'] == melhor_cluster_id].index[0]
                
                # Atualiza CPFs e contagem
                self.clusters_df.at[idx, 'cpfs_unicos'].add(cpf_atual)
                total_cpfs = len(self.clusters_df.at[idx, 'cpfs_unicos'])
                self.clusters_df.at[idx, 'quantidade_mensagens'] += 1
                
                # Verifica regra de Influencer
                if total_cpfs >= self.min_cpfs:
                    self.clusters_df.at[idx, 'e_influenciador'] = True
                
                # Atualiza o centroide (média móvel)
                n = self.clusters_df.at[idx, 'quantidade_mensagens']
                novo_centroide = (self.centroides_lista[idx] * (n-1) + vetor_atual.toarray()) / n
                self.centroides_lista[idx] = novo_centroide
            else:
                # Cria novo cluster
                novo_id = len(self.clusters_df) + 1
                novo_registro = {
                    'cluster_id': novo_id,
                    'quantidade_mensagens': 1,
                    'cpfs_unicos': {cpf_atual},
                    'e_influenciador': False
                }
                self.clusters_df = pd.concat([self.clusters_df, pd.DataFrame([novo_registro])], ignore_index=True)
                self.centroides_lista.append(vetor_atual.toarray())
                melhor_cluster_id = novo_id

            # Guardar resultado da linha atual
            status_influencer = self.clusters_df[self.clusters_df['cluster_id'] == melhor_cluster_id]['e_influenciador'].values[0]
            resultados_finais.append({
                'id': id_origem,
                'cluster_atribuido': melhor_cluster_id,
                'score_similaridade': maior_similaridade,
                'status_influenciador': 1 if status_influencer else 0
            })

        # Salva as alterações nos arquivos
        self._salvar_estado()
        
        # Une os resultados com o dataframe original para retorno
        df_resultados = pd.DataFrame(resultados_finais)
        return pd.merge(novos_dados, df_resultados, on='id')

# --- EXEMPLO DE USO EM PRODUÇÃO ---

if __name__ == "__main__":
    detector = DetectorIncrementalInfluenciador()

    # Se for a primeira vez rodando na vida:
    if detector.vetorizador is None:
        base_treino = [
            "O pix está fora do ar desde cedo",
            "O pix está fora do ar desde cedo",
            "O pix está fora do ar desde cedo",
            "O pix está fora do ar desde cedo",
            "O pix está fora do ar desde cedo",
            "O pix está fora do ar desde cedo",
            "Não consigo realizar transferencia via pix",
            "Meu cartão de crédito ainda não chegou", 
            "Bom dia, gostaria de tirar uma duvida",
            "O aplicativo é muito bonito parabens", 
            "O aplicativo é muito bonito", 
            "Alguém me ajuda com o pix por favor",
            "Alguém me ajuda com o pix",
            "me ajuda com o pix por favor",
        ]
        detector.inicializar_modelo(base_treino)

    # Simulação de dados chegando hoje
    dados_hoje = pd.DataFrame({
    'id': [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
    'texto': [
        "Erro generalizado no pix, arrumem logo",
        "Pix travado aqui também",
        "Estou aguardando a entrega do cartão a dias",
        "Atendimento péssimo ninguem responde",
        "Vocês são horriveis me atendam", 
        "Vou processar voces pelo atendimento ruim",
        "O aplicativo é muito bonito", 
        "Alguém me ajuda com o pix por favor",
        "Alguém me ajuda com o pix",
        "me ajuda com o pix por favor",
        "O pix está fora do ar desde cedo"

    ],
    'cpf': ['666', '222', '777', '888', '897', '888', '234', '345', '667', '245', '756']
    })

    relatorio = detector.processar_lote_diario(dados_hoje)
    print("\n--- RESULTADO DO PROCESSAMENTO ---")
    print(relatorio)


--- RESULTADO DO PROCESSAMENTO ---
    id                                        texto  cpf  cluster_atribuido  \
0    7       Erro generalizado no pix, arrumem logo  666                 13   
1    8                      Pix travado aqui também  222                 14   
2    9  Estou aguardando a entrega do cartão a dias  777                 15   
3   10         Atendimento péssimo ninguem responde  888                 16   
4   11               Vocês são horriveis me atendam  897                 17   
5   12    Vou processar voces pelo atendimento ruim  888                 18   
6   13                  O aplicativo é muito bonito  234                 19   
7   14          Alguém me ajuda com o pix por favor  345                 20   
8   15                    Alguém me ajuda com o pix  667                 21   
9   16                 me ajuda com o pix por favor  245                 22   
10  17             O pix está fora do ar desde cedo  756                 23   

    score_simil

In [13]:
# --- SIMULAÇÃO DE CARGA DE DADOS REAIS ---

# 1. Supondo que você carregou seus dados do banco/CSV
# df_influ = pd.read_sql("SELECT protocolo, texto_cliente, cpf, data_abertura FROM tabela", conexao)

# 2. Carregar o histórico de protocolos já processados (para não repetir trabalho)
caminho_processados = "protocolos_finalizados.csv"
if os.path.exists(caminho_processados):
    protocolos_antigos = pd.read_csv(caminho_processados)['protocolo'].tolist()
else:
    protocolos_antigos = []

# 3. Filtrar apenas o que é NOVO (O seu "Anti-Join")
df_novos_casos = df_influ[~df_influ['protocolo'].isin(protocolos_antigos)].copy()

# 4. Ajustar nomes de colunas para o Detector (ele espera 'id', 'texto', 'cpf')
df_novos_casos = df_novos_casos.rename(columns={
    'protocolo': 'id',
    'texto_cliente': 'texto',
    'cpf': 'cpf' # Certifique-se que a coluna CPF existe no seu df_influ
})

if not df_novos_casos.empty:
    detector = DetectorIncrementalInfluenciador()

    # Caso seja a PRIMEIRA execução de todas (Cold Start)
    if detector.vetorizador is None:
        print("Iniciando primeira execução histórica...")
        detector.inicializar_modelo(df_novos_casos['texto'])

    # Processamento Incremental
    print(f"Processando {len(df_novos_casos)} novos protocolos...")
    relatorio_final = detector.processar_lote_diario(df_novos_casos)

    # 5. Salvar a lista de protocolos processados para o dia de amanhã
    novos_ids = pd.DataFrame({'protocolo': df_novos_casos['id']})
    novos_ids.to_csv(caminho_processados, mode='a', index=False, header=not os.path.exists(caminho_processados))
    
    print("Processamento concluído e salvo.")
else:
    print("Não há novos dados para processar hoje.")

NameError: name 'df_influ' is not defined

In [ ]:
# --- SIMULAÇÃO COM DADOS MAIS COMPLEXOS ---

dados_dia_1 = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'texto': [
        "O pix está fora do ar desde cedo",
        "Não consigo realizar transferencia via pix",
        "Meu cartão de crédito ainda não chegou", 
        "Bom dia, gostaria de tirar uma duvida",
        "O aplicativo é muito bonito parabens", 
        "O aplicativo é muito bonito", 
        "Alguém me ajuda com o pix por favor",
        "Alguém me ajuda com o pix",
        "me ajuda com o pix por favor",
    ],
    'cpf': ['111', '222', '333', '444', '555', '111', '098', '356', '356']
})

dados_dia_2 = pd.DataFrame({
    'id': [7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
    'texto': [
        "Erro generalizado no pix, arrumem logo",
        "Pix travado aqui também",
        "Estou aguardando a entrega do cartão a dias",
        "Atendimento péssimo ninguem responde",
        "Vocês são horriveis me atendam", 
        "Vou processar voces pelo atendimento ruim",
        "O aplicativo é muito bonito", 
        "Alguém me ajuda com o pix por favor",
        "Alguém me ajuda com o pix",
        "me ajuda com o pix por favor",
    ],
    'cpf': ['666', '222', '777', '888', '897', '888', '234', '345', '667', '245']
})

print(">>> INICIANDO SISTEMA >>>")
detector = IncrementalInfluencerDetector(similarity_threshold=0.6)

# Passo 1: Treinamento Inicial (Cold Start)
# Usamos o dia 1 para 'aprender' o vocabulário
texts_train = [TextPreprocessor.clean(t) for t in dados_dia_1['texto']]
detector.fit_vectorizer(texts_train)

# Passo 2: Processamento do Dia 1
print("\n>>> PROCESSANDO DIA 1 (Base Histórica)...")
resultado_d1 = detector.process_daily_batch(dados_dia_1)
print(resultado_d1[['texto', 'cluster_id', 'is_influencer', 'action_log']])

# Passo 3: Processamento do Dia 2 (Incremental)
print("\n>>> PROCESSANDO DIA 2 (Novos Casos)...")
# Note que o CPF 555 vai se juntar ao cluster do saque.
# Como teremos CPFs 111, 333 e 555 no cluster, ele virará Influencer = True
resultado_d2 = detector.process_daily_batch(dados_dia_2)
print(resultado_d2[['texto', 'cluster_id', 'is_influencer', 'action_log']])

# Verificar estado final do Cluster que virou Influencer
cluster_saque_id = resultado_d2.iloc[0]['cluster_id']
print(f"\n>>> ESTADO FINAL CLUSTER {cluster_saque_id} <<<")
print(f"CPFs únicos: {detector.clusters[cluster_saque_id]['cpfs']}")
print(f"É Influencer? {detector.clusters[cluster_saque_id]['is_influencer']}")

>>> INICIANDO SISTEMA >>>
--- Treinando Vectorizer com 9 textos iniciais ---

>>> PROCESSANDO DIA 1 (Base Histórica)...
                                        texto  cluster_id  is_influencer  \
0            O pix está fora do ar desde cedo           1          False   
1  Não consigo realizar transferencia via pix           2          False   
2      Meu cartão de crédito ainda não chegou           3          False   
3       Bom dia, gostaria de tirar uma duvida           4          False   
4        O aplicativo é muito bonito parabens           5          False   
5                 O aplicativo é muito bonito           5          False   
6         Alguém me ajuda com o pix por favor           6          False   
7                   Alguém me ajuda com o pix           6          False   
8                me ajuda com o pix por favor           6          False   

         action_log  
0       New Cluster  
1       New Cluster  
2       New Cluster  
3       New Cluster  
4       N